In [1]:
import torch
import numpy as np
from abc import ABC, abstractmethod

In [2]:
def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width

In [3]:
class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass

In [4]:
def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)

In [5]:
class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)

# Реализация свертки с помощью циклов

In [6]:
def generate_input_data():
    n_kernel = int(input('Введите количество фильтров '))
    in_channels = int(input('Введите количество слоев в фильтре '))
    kernel_size = int(input('Введите размер фильтра '))
    
    kernel = torch.FloatTensor(np.random.randint(1, 100, n_kernel*in_channels*kernel_size*kernel_size))\
             .reshape(n_kernel, in_channels, kernel_size, kernel_size)
    
    batch_size = int(input('Введите размер батча '))
        
    while True:
        input_height = int(input('Введите высоту картинки '))
        input_width = int(input('Введите ширину картинки '))
        if (input_height >= kernel_size) and (input_width >= kernel_size):
            break
        else:
            print('Размер картинки должен быть больше размера фильтра. Повторите ввод.')
    
    input_tensor = torch.FloatTensor(np.random.randint(1, 100,  batch_size * in_channels * input_height * input_width))\
             .reshape(batch_size, in_channels, input_height, input_width)
    
    stride = int(input('Введите шаг свертки '))
    
    return kernel, input_tensor, batch_size, stride, input_height, input_width, in_channels

In [19]:
# Сверточный слой через циклы.
class Conv2dLoop(ABCConv2d):
    def __call__(self, input_tensor):
        output_tensor = torch.zeros(calc_out_shape(input_tensor.shape, self.out_channels, self.kernel_size, self.stride,
                                                    padding=0))
        for item in range(output_tensor.shape[0]):
            for channel in range(self.out_channels):
                for i in range(0, output_tensor.shape[2], self.stride):
                    for j in range(0, output_tensor.shape[3], self.stride):
                        output_tensor[item][channel][i][j] = (self.kernel[channel] * 
                                                 input_tensor[item, :, i:i+self.kernel_size, j:j+self.kernel_size]).sum()
        return output_tensor

In [20]:
def test_conv2d_layer(conv2d_layer_class):
    kernel, input_tensor, batch_size, stride, input_height, input_width, in_channels = generate_input_data()

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)

In [21]:
print(test_conv2d_layer(Conv2dLoop))

Введите количество фильтров 1
Введите количество слоев в фильтре 3
Введите размер фильтра 3
Введите размер батча 2
Введите высоту картинки 4
Введите ширину картинки 4
Введите шаг свертки 2
True


# Реализация свертки с помощью матричного умножения 

In [27]:
class Conv2dMatrix(ABCConv2d):
    def _unsqueeze_kernel(self, torch_input, output_height, output_width):
        _, in_channels, in_height, in_width = torch_input.shape
        ku_size = [self.out_channels, output_height, output_width, in_channels, in_height, in_width]
        kernel_unsqueezed = torch.zeros(ku_size, dtype=torch.float32)
        for i in range(output_height):
            for j in range(output_width):
                h_slice = slice(i*self.stride, i*self.stride+self.kernel_size)
                w_slice = slice(j*self.stride, j*self.stride+self.kernel_size)
                kernel_unsqueezed[:, i, j, :, h_slice, w_slice] = self.kernel.type(torch.float32)
        return kernel_unsqueezed.view(-1, in_channels*in_height*in_width)

    def __call__(self, torch_input):
        batch_size, out_channels, output_height, output_width\
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.kernel.shape[0],
                kernel_size=self.kernel.shape[2],
                stride=self.stride,
                padding=0)

        kernel_unsqueezed = self._unsqueeze_kernel(torch_input, output_height, output_width)
        result = kernel_unsqueezed @ torch_input.view((batch_size, -1)).permute(1, 0)
        return result.permute(1, 0).view((batch_size, self.out_channels,
                                          output_height, output_width))

In [28]:
print(test_conv2d_layer(Conv2dMatrix))

Введите количество фильтров 2
Введите количество слоев в фильтре 3
Введите размер фильтра 4
Введите размер батча 5
Введите высоту картинки 5
Введите ширину картинки 5
Введите шаг свертки 1
True
